# Notebook for Toronto Neighborhoods project

In [32]:
# imports
import pandas as pd
import lxml
import requests

### 1. Read table from wikipedia

In [3]:
# get table from wikipedia
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
neigh = tables[0]
neigh.rename(columns = {'Postal Code': 'PostalCode'}, inplace=True)

In [4]:
# subset to Borough != 'Not assigned'
mask = neigh.Borough != 'Not assigned'
neigh = neigh[mask].reset_index(drop=True)
neigh.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# check for Neighbourhood == 'Not assigned'
print(neigh[neigh.Neighbourhood == 'Not assigned'].shape)
print('All Neighbourhoods assigned!')

(0, 3)
All Neighbourhoods assigned!


In [6]:
# check for duplicate PostalCodes
print(neigh['PostalCode'].value_counts().max())
print('No duplicate PostalCodes found!')

1
No duplicate PostalCodes found!


In [6]:
# print the dataframe shape
print(neigh.shape)

(103, 3)


### 2. Add geocoding info to the neighborhoods

In [27]:
# note: the geocoder package did not work so will use the CSV file
geo = pd.read_csv('..\..\data\Geospatial_Coordinates.csv')
geo.rename(columns = {'Postal Code': 'PostalCode'}, inplace=True)
neigh_geo = neigh.merge(geo, on='PostalCode')
neigh_geo.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### 3. Analyze and cluster neighborhoods

In [28]:
CLIENT_ID = 'C4CUJ3LORAQARFUCBLZVSVWV5U4IKEHHNF42IKYBVOBOJPFW' # your Foursquare ID
CLIENT_SECRET = '1WKTMZ3NSZW3HEBCECOJLYPDUTXONYH3EQNQV2WHIKVNAM2Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C4CUJ3LORAQARFUCBLZVSVWV5U4IKEHHNF42IKYBVOBOJPFW
CLIENT_SECRET:1WKTMZ3NSZW3HEBCECOJLYPDUTXONYH3EQNQV2WHIKVNAM2Z


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names=neigh_geo['PostalCode'],
                                 latitudes=neigh_geo['Latitude'],
                                 longitudes=neigh_geo['Longitude']
                                )

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636
